# Ensemble methods

In this notebook, you will employ **boosting** and **stacking** to combine multiple models into a (hopefully) better one. Here's what you will do.
 - [Part 1](#1.-Dataset) - Feature engineering
 - [Part 2](#2.-Baseline-models) - Baseline models
 - [Part 3](#3.-Gradient-boosting) - Gradient boosting
 - [Part 4](#4.-Stacking) - Stacking

Let's first import all the packages that you will need during this assignment.
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [scipy](https://www.scipy.org) provides many numerical routines for scientific computing.
- [sklearn](http://scikit-learn.org/stable/) provides simple and efficient tools for data analysis and machine learning.
- [xgboost](https://github.com/dmlc/xgboost) is an highly-optimized library for gradient boosting.
- [pandas](https://pandas.pydata.org) is a library providing easy-to-use data structures and data analysis tools.
- [matplotlib](http://matplotlib.org) is a library for plotting graphs in Python.
- [seaborn](https://seaborn.pydata.org/index.html) provides a high-level interface for drawing attractive statistical graphics.

In [ ]:
import numpy as np
import xgboost as xgb
import pandas as pd
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
from scipy.stats import norm, skew

from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_predict
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.base import clone

np.random.seed(42)

# 1. Dataset

The Ames Housing dataset describes the sale of individual residential property in Ames, Iowa from 2006 to 2010. It is a modernized and expanded version of the often cited Boston Housing dataset. More information can be found [here](https://ww2.amstat.org/publications/jse/v19n3/decock.pdf).

The Ames dataset contains 2930 observations and 80 variables (plus 2 observation identifiers) of different type: 
 - 23 nominal
 - 23 ordinal
 - 14 discrete 
 - 20 numerical.
 
The goal is to build a regression model for the variable `SalePrice`. 

Let's give a quick look at the data.

In [ ]:
data = pd.read_csv("data/AmesHousing.csv", sep=";")

data.head()

It seems that the dataset needs some feature engeneering. Let's start with a simple task.

---
**Exercise:** Remove the columns `Order` and `PID` from the dataset.

**Hint:** Use the pandas function `drop()`.

In [ ]:
### START CODE HERE ### (≈ 1 line)

### END CODE HERE ###



In [ ]:
data.head()

In [ ]:
print("Dataset size: " + str(data.shape))

**Expected Output**:
       
<table style="width:20%">
  
  <tr>
    <td>**Dataset size**</td>
    <td> (2930, 80) </td> 
  </tr>
  
</table>

In [ ]:
assert data.shape == (2930,80)

## 1.1 Missing data 

Missing data occur when no data value is stored for a variable in an observation. This is a common occurrence in real-world datasets, and it can have a significant effect on the conclusions that can be drawn from the data. Understanding the reasons why data are missing is important to correctly handle the remaining data. 

The following code computes the amount of missing data for each variable in the dataset.

In [ ]:
# missing values
total = data.isnull().sum().sort_values(ascending=False)

# show
pd.DataFrame(data={'Missing': total}).head(30)

By reading the data description, we can understand that missing values actually have a precise meaning for some features:
 - **PoolQC**: NA means *No Pool*.
 - **MiscFeature**: NA means *No Misc Feature*.
 - **Alley**: : NA means *No Alley*.
 - **Fence**: : NA means *No Fence*.
 - **FireplaceQu** : NA means *No Fireplace*.
 - **GarageXXX**: NA means *No Garage*.
 - **BsmtXXX**: NA means *No Basement*.
 - **MasVnrXXX**: NA means *No masonry veneer*.

For other features, missing data can be replaced with "default" values:
 - **LotFrontage** : Since the area of each street connected to the house property most likely have a similar area to other houses in its neighborhood , we can fill in missing values by the median `LotFrontage` of the neighborhood.
 - **Electrical** : It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.

---
**Exercise:** Replace missing data with the appropriate "default" values. 
 
 - Use the string "None" in `PoolQC`, `MiscFeature`, `Alley`, `Fence`, `FireplaceQu`.
     
 - Use the string "None" in `GarageType`, `GarageFinish`, `GarageQual`, `GarageCond`.
  
 - Use the value 0 in `GarageYrBlt`, `GarageArea`, `GarageCars`.
 
 - Use the string "None" in `BsmtQual`, `BsmtCond`, `BsmtExposure`, `BsmtFinType1`, `BsmtFinType2`.
     
 - Use the value 0 in `BsmtFinSF1`, `BsmtFinSF2`, `BsmtUnfSF`, `TotalBsmtSF`, `BsmtFullBath`, `BsmtHalfBath`.
 
 - Use the string "None" in `MasVnrType` and the value 0 in `MasVnrArea`.
 
 - Use the most common value in `Electrical`.
   

**Hint:** Use the pandas function `fillna()` to replace NA with a given value; use the pandas function `mode()` to find the most common value of a column.

In [ ]:
none_features = ('PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 
                 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 
                 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType')
zero_features = ('GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 
                 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea')
cst_feature = 'Electrical'

### START CODE HERE ### (≈ 5 line)

### END CODE HERE ###



In [ ]:
total = data.isnull().sum().sort_values(ascending=False)
pd.DataFrame(data={'Missing': total}).head(2)

**Expected Output**:
       
<table style="width:30%">
  
  <tr>
    <td>**LotFrontage**</td>
    <td> 490 </td> 
  </tr>
  
  <tr>
    <td>**SalePrice**</td>
    <td> 0 </td> 
  </tr>

</table>

In [ ]:
for col in (none_features + zero_features + (cst_feature,)):
    assert data[col].isnull().sum() == 0

---
**Exercise:** Replace the missing data in `LotFrontage` with the following procedure.
 1. Group the values in `LotFrontage` having the same `neighborhood`.
 - Compute the median value of `LotFrontage` on each group separately.
 - For each group, replace the missing values in `LotFrontage` with the corresponding median.
  
**Hint:** Use the pandas function `groupby()` to group observations by the value of a column, and the function `transform()` to manipulate each group separately. See this [tutorial](http://pbpython.com/pandas_transform.html) for more information about groups in pandas.

In [ ]:
### START CODE HERE ### (≈ 1 line)

### END CODE HERE ###



In [ ]:
total = data.isnull().sum().sort_values(ascending=False)
pd.DataFrame(data={'Missing': total}).head(2)

**Expected Output**:
       
<table style="width:30%">
  
  <tr>
    <td>**LotFrontage**</td>
    <td> 3 </td> 
  </tr>
  
  <tr>
    <td>**SalePrice**</td>
    <td> 0 </td> 
  </tr>

</table>

In [ ]:
assert data['LotFrontage'].isnull().sum() == 3

With the previous approach, we weren't able to determine `LotFrontage` for 3 houses, as all the observations in the corresponding neighborhoods have missing values in `LotFrontage`. To avoid the risk of introducing a bias in the data, we decide to drop these observations.

In [ ]:
mask = data['Neighborhood'].isin(["Landmrk","GrnHill"])
data = data[~mask]

print("Missing values: " + str(data.isnull().sum().sum()))
print("Dataset shape: " + str(data.shape))

## 1.2 Correlation matrix

Now, it's time to analyze the correlation between `SalePrice` and the other variables.

In [ ]:
# correlation matrix
corrmat = data.corr()

# variables for heatmap
k = 10 
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = corrmat[cols].loc[cols]

# show
sns.set(font_scale=1.5)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

From the top-10 correlation matrix, we can deduce that:

- `OverallQual`, `GrLivArea` and `TotalBsmtSF` are strongly correlated with `SalePrice`.
- `GarageCars` is correlated to `GarageArea`. This seems reasonable, as the number of cars fitting into a garage is proportional to the garage area.
- `TotalBsmtSF` is correlated to `1stFlrSF`. This seems reasonable, as the floors of an house are usually the same size.

Now, let's proceed to the scatter plots between `SalePrice` and the most correlated variables.

In [ ]:
sns.set()

cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF']
sns.pairplot(data[cols], size = 2.5)
plt.show()

The variables `OverallQual`, `GrLivArea` and `TotalBsmtSF` seem to be linearly related with `SalePrice`. The relationships are positive, which means that as one variable increases, the other also increases.

Since area-related features are very important to determine house prices, we are going to craft a new feature that indicates the total area of basement and above ground living space.

---
**Exercise:** Add a new feature `TotalSF` given by the sum of `GrLivArea` and `TotalBsmtSF`.

In [ ]:
new_feature = 'TotalSF'

### START CODE HERE ### (≈ 1 line)

### END CODE HERE ###



In [ ]:
print("Dataset shape: " + str(data.shape))

**Expected Output**:
       
<table style="width:30%">
  
  <tr>
    <td>**Dataset shape**</td>
    <td> (2927, 81) </td> 
  </tr>

</table>

In [ ]:
assert data.shape == (2927, 81)
assert 'TotalSF' in data.columns

## 1.3 Outliers

Outliers are something that we should be aware of. Why? Because outliers can markedly affect our models, and can be a valuable source of information, providing us insights about specific behaviours. Outlier handling is a complex subject. Here, we'll just do a quick analysis through the scatter plots.

Let's plot `SalePrice` versus `GrLivArea`, so as to easily spot any large houses that sold for really cheap.

In [ ]:
plt.scatter(data.GrLivArea, data.SalePrice)
plt.title("Looking for outliers")
plt.xlabel("GrLivArea")
plt.ylabel("SalePrice")
plt.show()

Here's what we can deduce from the scatterplot.

- The three values with bigger `GrLivArea` seem strange and they are not following the crowd. Maybe they refer to agricultural area and that could explain the low price. Even though we are not sure about this, it seems that these points are not representative of the typical case. Therefore, we'll define them as outliers and delete them.
 
- The two observations in the top of the plot (above 700'000) are also suspect. They look like two special cases, however they seem to be following the trend. For that reason, we will keep them.

Outliers removal is not always safe. The points that we have decided to delete are very huge and really bad (extremely large areas for very low prices). There are probably others outliers in the training data. However, removing all of them may affect badly our models if ever there were also outliers in new data. That's why, instead of removing them all, we will just manage to make some of our models robust on them. 

---

**Exercise:** Remove the outliers in which `SalePrice` is less than 300'000 and `GrLivArea` is greater than 4'000.

In [ ]:
### START CODE HERE ### (≈ 1 line)

### END CODE HERE ###



In [ ]:
fig, ax = plt.subplots()
ax.scatter(data['GrLivArea'], data['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

In [ ]:
print("Dataset shape: " + str(data.shape))

**Expected Output**:
       
<table style="width:30%">
  
  <tr>
    <td>**Dataset shape**</td>
    <td> (2924, 81) </td> 
  </tr>
  
</table>

In [ ]:
assert data.shape == (2924, 81)

## 1.4 Categorical variables

The dataset contains different types of variable.
 - **Nominal** - This variable can take a fixed number of values. 
 - **Ordinal** - This variable can take a fixed number of values, and such values have a logical order.
 - **Discrete** - This variable can take any integer value.
 - **Numerical** - This variable can take any numerical value.

Let's start by grouping the variables according to their type.

In [ ]:
nominal = ['MSSubClass', 'MSZoning', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 
           'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
           'Heating', 'CentralAir', 'Electrical', 'GarageType', 'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition'] 
           
ordinal = ['LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 
           'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 
           'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']
           
discrete = ['YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 
            'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'YrSold']

Ordinal variables are converted into discrete features.

In [ ]:
for col in ordinal:
    v = list(data[col].values)
    data[col] = LabelEncoder().fit_transform(v)

Nominal variables are converted into binary features.

In [ ]:
# cast to string
for col in nominal:
    data[col] = data[col].apply(str)

# convert to binary features
data = pd.get_dummies(data)

In [ ]:
data.head()

## 1.5 Dataset split

Finally, we can split the available data in train and test sets.

In [ ]:
y = np.array(data['SalePrice'])
X = np.array(data.drop('SalePrice', axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# 2. Baseline models

There exist many different approaches to deal with regression. In this notebook, we'll consider some of the models available in [scikit-learn](http://scikit-learn.org/stable/):
 - Linear regression
 - Regularized regression
 - Random forest.

## 2.1 Linear regression

Let's start with linear regression, which is the simplest model we could think of. The following code performs these steps:
 - The model is fit to the train set, 
 - The performance is evaluated on the test set.

In [ ]:
# build
linreg = LinearRegression()

# train
linreg.fit(X_train, y_train)

# predict
y_pred = linreg.predict(X_test)

# test
score = np.sqrt(mean_squared_error(y_test, y_pred))
    
# show
print('Linear regression: {:.2f}'.format(score))

To gain insight into the learned model, we can visualize the residual plot.

In [ ]:
sns.set_context("talk")

plt.figure(figsize=(10,5))
sns.residplot(y_pred, y_test, lowess=True, color="r", scatter_kws={"color":"g", "s":20, "alpha": 0.4})
plt.title("Residual plot")

We can observe that the residuals are **not** uniformely distributed around the zero (the red line is u-shaped). This is a clear indication that there is a **nonlinear** relationship between the target variable `SalePrice` and the other variables. To figure out what kind of nonlinearity it could be, we need to go deep and understand how `SalePrice` complies with the statistical assumptions that enable us to apply linear regression. The main assumption that we should test is **normality**. In this notebook, we'll just check univariate normality by looking at:
 - **Histogram**, which tells us about kurtosis and skewness.
 - **Normal probability plot**, in which data should closely follow the diagonal that represents the normal distribution.

In [ ]:
fig = plt.figure(figsize=(15,4))
fig.add_subplot(121)
sns.distplot(data['SalePrice'], fit=norm);
fig.add_subplot(122)
res = stats.probplot(data['SalePrice'], plot=plt)

We can see that the distribution of `SalePrice` deviates from the normal distribution and has positive skewness. A simple data transformation can solve the problem. This is one of the awesome things you can learn in statistical books: in case of positive skewness, a **logarithm** transformation usually works well. Here's how such a transformation works:

 - Train the model to predict `LogSalePrice = log(SalePrice + 1)`.
 
 - Then, revert to the actual prediction using the formula `y = exp(LogSalePrice) - 1`.
 
---

**Exercise:** Implement the log transformation of the target variable in such a way that can be applied to any regression model available in scikit-learn.

**Hint:** You can implement a class that wraps a scikit-learn model, and redefines the following methods:
 - `fit(X, y)` - Transform the target variable `y` into `z = log(y+1)`, and then call `fit(X, z)` on the wrapped model.
 - `predict(X)` - Obtain the prediction `z` by calling `predict(X)` on the wrapped model, and then return `exp(z)-1`
 
For numerical robustness, use the numpy functions `log1p` and `expm1` to perform the actual tranformations.

In [ ]:
class LogTransform:
        
    def fit(self, X, y):
        ### START CODE HERE ###
        None
        ### END CODE HERE ###
        
        return self
        
    def predict(self, X):
        ### START CODE HERE ###
        None
        ### END CODE HERE ###
        

    def __init__(self, model):
        self.model = model
        
    def score(self, X, y):
        y_pred = self.predict(X)
        return np.sqrt(mean_squared_error(y, y_pred))
    
    def set_params(self, **kwargs):
        self.model = kwargs['model']
        return self
        
    def get_params(self, deep=True):
        return {'model': self.model}

In [ ]:
# build
linreg = LogTransform(model=LinearRegression())

# train
linreg.fit(X_train, y_train)

# predict
y_pred = linreg.predict(X_test)

# test
score = np.sqrt(mean_squared_error(y_test, y_pred))
    
# show
print('Linear regression: {:.2f}'.format(score))

**Expected Output**:

<table style="width:30%">
  
  <tr>
    <td>**Linear regression**</td>
    <td> 19880.33 </td> 
  </tr>
  
</table>

In [ ]:
np.testing.assert_almost_equal(score, 19880.33, decimal=0)

---

Let's look again at the residual plot.

In [ ]:
plt.figure(figsize=(10,5))
sns.residplot(y_pred, y_test, lowess=True, color="r", scatter_kws={"color":"g", "s":20, "alpha": 0.4})
plt.title("Residual plot")

From the figure, we can see that the residuals are now regularly distributed around the zero. Hurrah!

## 2.2 Regularized regression

Other approaches worth exploring are ridge regression, lasso, and elastic net. They are all linear regression models, differing from each other in the regularization term (used to reduce the risk of over-fitting).
 - **Ridge** - L2 regularization.
 - **Lasso** - L1 regularization.
 - **Elastic net** - Both.

Regularization works best when the data are normalized. The code below uses the Scikit-Learn method `make_pipeline` to define a pipeline in which the data are properly normalized before training and testing.

In [ ]:
ridge = make_pipeline(RobustScaler(), LogTransform(model=Ridge(random_state=42)))
lasso = make_pipeline(RobustScaler(), LogTransform(model=Lasso(random_state=42)))
elnet = make_pipeline(RobustScaler(), LogTransform(model=ElasticNet(random_state=42)))

models = (ridge, lasso, elnet)
names  = ('Ridge', 'Lasso', 'Elnet')

for clf, name in zip(models, names):
    
    # train
    clf.fit(X_train, y_train)
    
    # predict
    y_pred = clf.predict(X_test)
    
    # test
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # show
    print('{:s}: {:.2f}'.format(name, score))

You can see that the performances of lasso and elastic net are not satisfactory. This happens because such models depend on hyper-parameters that control how much regularization is applied during training. In order to find the hyper-parameters leading to the best performance, you can use cross-validation. 

---

**Exercise:** Find the best hyper-parameters via cross-validation. **You are only allowed to use the train set.**
 
**Hint:** The following table reports the candidate values that you should test out with cross-validation. 
 
 <table style="width:60%"> 
  <tr>
    <td></td>
    <td>**Hyper-parameter**</td>
    <td style="width:60%">**Values**</td> 
  </tr> 
  <tr>
    <td>**Ridge**</td>
    <td>*alpha*</td>
    <td>1, 5, 10, 15, 30, 50</td> 
  </tr>
  <tr>
    <td>**Lasso**</td>
    <td>*alpha*</td>
    <td>0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1</td> 
  </tr> 
  <tr>
    <td>**Elastic net**</td>
    <td>*alpha*</td>
    <td>0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1</td> 
  </tr>
  <tr>
    <td></td>
    <td>*l1_ratio*</td>
    <td>0.2, 0.4, 0.6, 0.8, 1</td> 
  </tr>
</table>

To save time, you can adopt a coarse-to-fine approach.

In [ ]:
### START CODE HERE ### 
ridge_alphas = (1,)  # TODO: Add candidate values
lasso_alphas = (1,)
elnet_alphas = (1,)
elnet_l1     = (1,)
### END CODE HERE ###


# cross-validation strategy
cv = KFold(n_splits=5, random_state=42)

# models
ridge_cv = RidgeCV(alphas=ridge_alphas, cv=cv)
lasso_cv = LassoCV(alphas=lasso_alphas, cv=cv, random_state=42)
elnet_cv = ElasticNetCV(alphas=elnet_alphas, l1_ratio=elnet_l1, max_iter=5000, cv=cv, random_state=42)

# pipelines
ridge = make_pipeline(RobustScaler(), LogTransform(model=ridge_cv))
lasso = make_pipeline(RobustScaler(), LogTransform(model=lasso_cv))
elnet = make_pipeline(RobustScaler(), LogTransform(model=elnet_cv))

# train
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)
elnet.fit(X_train, y_train)
    
# print
print('Ridge: {:.2f}'.format(ridge.score(X_test,y_test)))
print('Lasso: {:.2f}'.format(lasso.score(X_test,y_test)))
print('Elnet: {:.2f}'.format(elnet.score(X_test,y_test)))
print('-----')
print('Best hyper-params (ridge): alpha={:2.2f}'.format(ridge_cv.alpha_))
print('Best hyper-params (lasso): alpha={:2.5f}'.format(lasso_cv.alpha_))
print('Best hyper-params (elnet): alpha={:2.5f}, l1_ratio={:2.2f}'.format(elnet_cv.alpha_, elnet_cv.l1_ratio_))

**Expected Output**:

<table style="width:30%">
  
  <tr>
    <td>**Ridge**</td>
    <td> 18836.40 </td> 
  </tr>
  
  <tr>
    <td>**Lasso**</td>
    <td> 19044.91 </td> 
  </tr>
  
  <tr>
    <td>**Elnet**</td>
    <td> 18980.21 </td> 
  </tr>
  
</table>

In [ ]:
np.testing.assert_almost_equal(ridge.score(X_test,y_test), 18836.40, decimal=0)
np.testing.assert_almost_equal(lasso.score(X_test,y_test), 19044.91, decimal=0)
np.testing.assert_almost_equal(elnet.score(X_test,y_test), 18980.21, decimal=0)

---

Let's check the residual plot of elastic net.

In [ ]:
plt.figure(figsize=(10,5))
sns.residplot(elnet.predict(X_test), y_test, lowess=True, color="r", scatter_kws={"color":"g", "s":20, "alpha": 0.4})
plt.title("Residual plot")

## 2.3 Random forest

In the realm of nonlinear models, random forest is one of the most interesting techniques. The idea behind it is to build several decision trees independently, and then to average their predictions. In particular, each tree is learned from bootstrapped observations, using a random subset of the features at each node split. As a result of this randomness, the combined trees yield a prediction model that is usually better than any of the individual tree.

Random forest depends on the following hyper-parameters:
 - `n_estimators` - The number of trees in the forest.
 - `max_depth` - The maximum depth of each individual tree.
 - `max_features` - The number of features to consider when looking for the best split.
 - `min_samples_leaf` - The minimum number of samples required to be at a leaf node.
 
---

**Exercise:** Find the best hyper-parameters via cross-validation. **You are only allowed to use the train set.**
 
**Hint:** The following table reports the candidate values that you should test out with cross-validation.
 
 <table style="width:60%"> 
  <tr>
    <td>**Hyper-parameter**</td>
    <td style="width:60%">**Values**</td> 
  </tr> 
  <tr>
    <td>*n_estimators*</td>
    <td>60, 70, 80, 90, 100</td> 
  </tr>
  <tr>
    <td>*max_features*</td>
    <td>0.2, 0.4, 0.6, 0.8, 1</td> 
  </tr>
   <tr>
    <td>*max_depth*</td>
    <td>None</td> 
  </tr> 
  <tr>
    <td>*min_samples_leaf*</td>
    <td>1</td> 
  </tr>
</table>

To save time, you can adopt a coarse-to-fine approach.

In [ ]:
forest = RandomForestRegressor(random_state=42)

### START CODE HERE ### 
params_grid = { 
    "n_estimators": [10], # TODO: add candidate values
                          # TODO: add the other parameter and its candidate values
}
### END CODE HERE ###


# cross-validation strategy
cv = KFold(n_splits=5, random_state=42)

# hyper-parameter search
forest_cv = GridSearchCV(forest, params_grid, scoring='neg_mean_squared_error', cv=cv, verbose=1, n_jobs=4)
forest_cv.fit(X_train, y_train)

# best score
score = np.sqrt(-forest_cv.score(X_test,y_test))

In [ ]:
print('Random forest: {:.2f}'.format(score))
print('-----')
print('Best hyper-params: ' + str(forest_cv.best_params_))

**Expected Output**:

<table style="width:30%">
  
  <tr>
    <td>**Random forest**</td>
    <td> 20740.51 </td> 
  </tr>
  
  <tr>
    <td>**max_features**</td>
    <td> 0.4 </td> 
  </tr>
  
  <tr>
    <td>**n_estimators**</td>
    <td> 80 </td> 
  </tr>
  
</table>

In [ ]:
np.testing.assert_almost_equal(score, 20740.51, decimal=0)

---

Let's check the residual plot.

In [ ]:
plt.figure(figsize=(10,5))
sns.residplot(forest_cv.predict(X_test), y_test, lowess=True, color="r", scatter_kws={"color":"g", "s":20, "alpha": 0.4})
plt.title("Residual plot")

# 3. Gradient boosting

Gradient boosting is one of the most powerful techniques for learning an ensemble of simple models (e.g., decision trees). In this notebook, we'll use the rich and fast implementation provided by [XGBoost](https://github.com/dmlc/xgboost). 

In order to get the best performance out of XGBoost, some important hyper-parameters need to be carefully tuned.

 1. **Number of trees.** This parameter controls the ensemble size. In general, you want the number of trees to be as high as possible (>1000), but having it too high could lead to overfitting. It is usually set via early stopping.
 
 - **Learning rate.** This parameter scales the effect of each tree on the overall prediction. It is tightly coupled with the ensemble size: a smaller value of the learning rate will require a larger number of trees, and vice versa. Typically, a much better generalization error is obtained with small values (<0.1).
 
 - **Maximum tree depth**. This parameter controls the tree dept. Good values for it are generally between 1-10.
 
 - **Row & column subsampling**. These parameters allow to perform a random subsampling of observations and features (like random forests). Tipical values are between 0.5 and 1.
 
 - **L1 & L2 regularization**. These parameters allow to add a penalization on the weights used to combine the trees into the final prediction. Big values will make the model more conservative.

There are many other hyper-parameters in XGBoost. You can refer to [this page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md) for the full list. 

## 3.1 XGBRegressor

In this notebook, we'll use the Scikit-Learn wrapper interface of the XGBoost regression model: `XGBRegressor`. 

In [ ]:
# model
boost = xgb.XGBRegressor(seed=42)

# train
boost.fit(X_train, y_train)

# test
y_pred = boost.predict(X_test)
score = np.sqrt(mean_squared_error(y_test, y_pred))

# show
print('Gradient boosting: {:.2f}'.format(score))

Even with the default hyper-parameters, gradient boosting performs on par with baseline models. Let's see if we can do better.

## 3.2 Ensemble size

The number of trees in the ensemble is one of the hyper-parameters that control the tradeoff between **under-fitting** and **over-fitting**. To determine if the ensemble is overfitting, we can keep track of the performance over the train and test sets during the learning phase. So doing, we'll be able to monitor how the prediction evolves as we add more trees to the ensemble. 

The XGBoost model can evaluate the performance on one or more datasets during training. This can be done by invoking the method `fit()` with two additional arguments: a list of datasets and the evaluation metric. Some of the [metrics](http://xgboost.readthedocs.io/en/latest//parameter.html#learning-task-parameters) supported by XGBoost are:
 - `rmse` for root mean squared error
 - `mae` for mean absolute error
 - `logloss` for binary logarithmic loss 
 - `mlogloss` for multi-class cross-entropy loss
 - `error` for classification error
 - `auc` for area under ROC curve.

The following code shows how to keep track of RMSE on both the train and test sets. 

In [ ]:
# Define the ensemble
boost = xgb.XGBRegressor(n_estimators=500, learning_rate=0.5, seed=42)

# Prepare the evaluation sets
eval_set = [(X_train,y_train),(X_test,y_test)]

# Train the ensemble
boost.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)

Let's plot the train and test errors as a function of the number of trees. This is called **deviance plot**. 

In [ ]:
def deviance_plot(boost, ax=None, label='', train_color='#2c7bb6', test_color='#d7191c', alpha=1.0):
    
    results = boost.evals_result()
    train_scores = results['validation_0']['rmse']
    test_scores  = results['validation_1']['rmse']
    n_estimators = boost.n_estimators

    if ax is None:
        fig = plt.figure(figsize=(15,5))
        ax = plt.gca()
        
    ax.plot(np.arange(n_estimators) + 1, train_scores, color=train_color,  label='Train %s' % label, linewidth=2, alpha=alpha)
    ax.plot(np.arange(n_estimators) + 1, test_scores, color=test_color, label='Test %s' % label, linewidth=2, alpha=alpha)
    ax.set_ylabel('Error')
    ax.set_xlabel('Number of trees in the ensemble')
    ax.legend(loc='upper right')
    
    return ax

In [ ]:
ax = deviance_plot(boost)
ax.set_ylim(0,40000)

train_scores = boost.evals_result()['validation_0']['rmse']
test_scores = boost.evals_result()['validation_1']['rmse']
argmin = np.argmin(test_scores)
minim = np.min(test_scores)

unidir_arrow = {'xycoords': 'data', 'textcoords': 'data', 'arrowprops': {'arrowstyle': '->'}}
bidir_arrow  = {'xycoords': 'data', 'textcoords': 'data', 'arrowprops': {'arrowstyle': '<->'}}
ax.annotate('Lowest test error', xy=(argmin+1, minim), xytext=(argmin-20, minim+5000), **unidir_arrow)
n_est = boost.n_estimators - 50
ann = ax.annotate('', xy=(n_est, test_scores[n_est]), xytext=(n_est, train_scores[n_est]), **bidir_arrow)
ax.text(n_est+2, (test_scores[n_est+1] + train_scores[n_est+1])/2, 'Overfitting')

plt.show()

You can see that there is a **huge gap** between the train and the test errors. This is a clear sign that the model is **overfitting**. Indeed, after you add many trees to the ensemble, the model gets too much capacity (i.e., high variance) and starts fitting the idiosyncracies of the training data.

Fortunately, XGBoost provides some knobs to reduce overfitting:
 - Early stopping
 - Tree-depth control
 - Random subsampling
 - Regularization 

## 3.3 Early stopping

One of the possible ways to fight overfitting is **early stopping**. The idea behind it is to stop the training when the model begins to overfit. In practice, the training can be stopped when no improvement is observed on the test error after a fixed number of epochs. 

To activate the early stopping in XGBoost, you need to pass three arguments to the method `fit()`: 
 - `eval_set` - A list containing the test set (in the **last** position)
 - `eval_metric` - The performance metric
 - `early_stopping_rounds` - The number of epochs over which no improvement must be observed in order to stop.

In [ ]:
# train
boost.fit(X_train, y_train, early_stopping_rounds=10, eval_set=eval_set, eval_metric="rmse", verbose=False)

# print
print("Best score: {:2.2f}".format(boost.best_score))
print("Best tree limit: " + str(boost.best_ntree_limit))

## 3.4 Tree depth

The hyper-paremeter `max_depth` controls the depth of each individual tree in the ensemble. A big value of `max_depth` increases the variance (i.e., the model tends to over-fit), whereas a small value increases the bias (i.e., the model tends to under-fit).

The following plot illustrates the effect of this parameter on the performance.

In [ ]:
boost_ctrl = xgb.XGBRegressor(n_estimators=500, max_depth=1, learning_rate=0.5, seed=42)

# train
boost.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)
boost_ctrl.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)

# show
ax = deviance_plot(boost)
ax = deviance_plot(boost_ctrl, ax=ax, label='(max_depth=1)', train_color='#abd9e9', test_color='#fdae61')
ax.set_ylim(0,40000)

n_est = boost.n_estimators - 100
ax.annotate('Higher bias', xy=(n_est, boost_ctrl.evals_result()['validation_0']['rmse'][n_est]), xytext=(n_est-40, 5000), **unidir_arrow)
ax.annotate('Lower variance', xy=(n_est-50, boost_ctrl.evals_result()['validation_1']['rmse'][n_est]), xytext=(n_est-100, 10000), **unidir_arrow)

## 3.5 Learning rate

The hyper-parameter `learning_rate` scales the effect of each tree on the overall prediction. It is tightly coupled with the ensemble size (i.e., the hyper-parameter `n_estimators`). A smaller value of `learning_rate` allows the ensemble to generalize better to new unseen data, but it requires a higher value of `n_estimators` to get good results. However, if the latter is too high, the model starts to overfit.

The following plot illustrates the effect of this parameter on the performance.

In [ ]:
boost_ctrl = xgb.XGBRegressor(n_estimators=500, learning_rate=0.1, seed=42)

# train
boost_ctrl.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)

# show
ax = deviance_plot(boost)
ax = deviance_plot(boost_ctrl, ax=ax, label='(learning_rate=0.1)', train_color='#abd9e9', test_color='#fdae61')
ax.set_ylim(0,40000)

n_est = boost.n_estimators - 100
ax.annotate('Higher bias', xy=(n_est, boost_ctrl.evals_result()['validation_0']['rmse'][n_est]), xytext=(n_est-40, 4000), **unidir_arrow)
ax.annotate('Lower variance', xy=(n_est-30, boost_ctrl.evals_result()['validation_1']['rmse'][n_est]), xytext=(n_est-80, 14000), **unidir_arrow)

## 3.5 Random subsampling

The training of each individual tree in the ensemble can be controlled by three hyper-parameters:
 - `subsample` - Randomly collect a fraction of observations before growing each tree
 - `colsample_bytree` - Randomly select a fraction of variables before growing each tree
 - `colsample_bylevel` - Randomly select a fraction of variables before finding the best split node
   
Note that `subsample` and `learning_rate` interact with each other. When you set `subsample`, you should also lower `learning_rate` and increase `n_estimators`. Likewise, `colsample` interacts with `max_depth`. As a rule of thumb, a good value of `colsample` could be the square root of the total number of features. Also, keep in mind that feature sampling works better if there is a sufficient large number of features.

The following plot illustrates the effect of these parameters on the performance.

In [ ]:
boost = xgb.XGBRegressor(n_estimators=500, learning_rate=0.1, seed=42)
boost_ctrl = xgb.XGBRegressor(n_estimators=500, colsample_bylevel=0.1, learning_rate=0.1, seed=42)

# train
boost.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)
boost_ctrl.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)

# show
ax = deviance_plot(boost)
ax = deviance_plot(boost_ctrl, ax=ax, label='(sampling=0.1)', train_color='#abd9e9', test_color='#fdae61')
ax.set_ylim(0,40000)

n_est = boost.n_estimators - 100
ax.annotate('Higher bias', xy=(n_est, boost_ctrl.evals_result()['validation_0']['rmse'][n_est]), xytext=(n_est-40, 4000), **unidir_arrow)
ax.annotate('Lower variance', xy=(n_est-30, boost_ctrl.evals_result()['validation_1']['rmse'][n_est]), xytext=(n_est-80, 14000), **unidir_arrow)

## 3.6 Regularization

The weights used to combine the trees into the final prediction can controlled by two hyper-parameters:
 - reg_lambda - Add a L2 regularization
 - reg_alpha - Add a L1 regularization 
 
The following plot illustrates the effect of these parameters on the performance.

In [ ]:
boost = xgb.XGBRegressor(n_estimators=500, learning_rate=0.5, reg_lambda=0, seed=42)
boost_ctrl = xgb.XGBRegressor(n_estimators=500, learning_rate=0.5, reg_lambda=10, seed=42)

# train
boost.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)
boost_ctrl.fit(X_train, y_train, eval_set=eval_set, eval_metric="rmse", verbose=False)

# show
ax = deviance_plot(boost, label='(lambda=0)')
ax = deviance_plot(boost_ctrl, ax=ax, label='(lambda=10)', train_color='#abd9e9', test_color='#fdae61')
ax.set_ylim(0,40000)

n_est = boost.n_estimators - 100
ax.annotate('Lower variance', xy=(n_est, boost_ctrl.evals_result()['validation_1']['rmse'][n_est]), xytext=(n_est-80, 14000), **unidir_arrow)

## 3.7 Hyper-parameter tuning


To obtain the best performance, you have to set hyper-parameters via cross-validation. Here is a good recipe:
 1. Pick `n_estimators` as large as computationally possible
 - Activate early stopping on the test set
 - Use cross-validation to find the best `max_depth`, `learning_rate`, `subsample`, `colsample`, and `reg_lambda`
 - Retrain with the best hyper-parameters and early stopping.
 
--- 

**Exercise:** Find the best-hyperparameters via cross-validation. **You are only allowed to use the train set.**
 
**Hint:**
 - You may find useful the Scikit-Learn class `GridSearchCV`.
 - The following table reports the values that are typically tested out with cross-validation 
 
 <table style="width:25%"> 
  <tr>
    <td>**Hyper-parameter**</td>
    <td>**Values**</td> 
  </tr> 
  <tr>
    <td>*n_estimators*</td>
    <td>1000</td> 
  </tr>
  <tr>
    <td>*learning_rate*</td>
    <td>0.001, 0.01, 0.1</td> 
  </tr> 
  <tr>
    <td>*max_depth*</td>
    <td>1, 2, 3, 4, 5</td> 
  </tr>
  <tr>
    <td>*subsample*</td>
    <td>0.2, 0.4, 0.6, 0.8, 1</td> 
  </tr>
  <tr>
    <td>*colsample*</td>
    <td>0.2, 0.4, 0.6, 0.8, 1</td> 
  </tr>
  <tr>
    <td>*reg_lambda*</td>
    <td>0.1, 1, 10</td> 
  </tr>
</table>

- To save time, you can adopt a coarse-to-fine approach.

In [ ]:
# early stopping
fit_params = {"early_stopping_rounds": 10, 
              "eval_metric": "rmse", 
              "eval_set": [(X_test, y_test)],
              "verbose": False}

### START CODE HERE ###
# grid of parameters - a dictionary with entries: "parameter name": [value1, values2, ...]. 
paramGrid = { }
### END CODE HERE ###


# ensemble definition
boost = xgb.XGBRegressor(n_estimators=1000)

# cross-validation strategy
cv = KFold(n_splits=5, random_state=42)

# hyper-parameter search
gridsearch = GridSearchCV(boost, paramGrid, cv=cv, verbose=2, n_jobs=4)
gridsearch.fit(X_train, y_train, **fit_params)

In [ ]:
print('Best hyperparameters: ' + str(gridsearch.best_params_))

**Expected Output**:

<table style="width:30%">
  
  <tr>
    <td>**learning_rate**</td>
    <td> 0.01 </td> 
  </tr>
  
  <tr>
    <td>**max_depth**</td>
    <td> 3 </td> 
  </tr>
  
  <tr>
    <td>**colsample_bytree**</td>
    <td> 0.4 </td> 
  </tr>
  
  <tr>
    <td>**subsample**</td>
    <td> 0.5 </td> 
  </tr>
  
  <tr>
    <td>**reg_lambda**</td>
    <td> 1 </td> 
  </tr>
  
</table>

In [ ]:
assert gridsearch.best_params_['learning_rate'] == 0.01
assert gridsearch.best_params_['max_depth'] == 3
assert gridsearch.best_params_['colsample_bytree'] == 0.4
assert gridsearch.best_params_['subsample'] == 0.5
assert gridsearch.best_params_['reg_lambda'] == 1

---

You can now retrain the ensemble using the best hyper-parameters.

In [ ]:
# model
boost = xgb.XGBRegressor(n_estimators=1000, **gridsearch.best_params_)

# train
boost.fit(X_train, y_train, **fit_params)

# test
y_pred = boost.predict(X_test, ntree_limit=boost.best_ntree_limit)

# score
score = np.sqrt(mean_squared_error(y_test, y_pred))

# print
print('Gradient boosting: {:.2f} (with {:d} trees)'.format(score, boost.best_ntree_limit))

Let's also check the residual plot.

In [ ]:
plt.figure(figsize=(10,5))
sns.residplot(y_pred, y_test, lowess=True, color="r", scatter_kws={"color":"g", "s":20, "alpha": 0.4})
plt.title("Residual plot")

# 4. Stacking

Stacked generalization is a way of combining multiple models through the concept of a meta learner. The general procedure is as follows:
- **Base models.** Several models are trained to generate an intermediate set of features (e.g., their predictions).
- **Meta learner.** A separate model is then trained to combine the intermediate features into the final prediction.

The tricky part is figuring out how to train the meta learner. Here's how it could be done.
1. Split the train set into two disjoint sets.
2. Train the base models on the first part.
3. Use them to generate the intermediate predictions on the second part.
4. Train the meta-learner using the intermediate predictions as the inputs, and the correct responses as the outputs.

The above procedure is practically the same as **hold-out** cross-validation, but it is also possible to use the **K-fold** approach (more on this later).

## 4.1 Stacked average

To gain more insight into the matter, let's begin with a simplyfied form of stacking, where the meta learner is just the average of the intermediate predictions, which has the advantage of requiring no training at all. The following code illustrates how to do so.

In [ ]:
# base models
models = [
    LogTransform(model=LinearRegression()),
    make_pipeline(RobustScaler(), LogTransform(Ridge(alpha=20))),
    make_pipeline(RobustScaler(), LogTransform(Lasso(alpha=0.0005, random_state=42))),
    make_pipeline(RobustScaler(), LogTransform(ElasticNet(alpha=0.001, l1_ratio=0.40, random_state=42))),
    RandomForestRegressor(n_estimators=80, max_features=0.4, random_state=42),
    xgb.XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=3, subsample=0.5, colsample_bytree=0.4, seed=42)
]

y_stacked = np.zeros([len(y_test), len(models)+1])

# intermediate predictions
for i,m in enumerate(models):
    
    # train
    m.fit(X_train, y_train)
    
    # test
    y_stacked[:,i] = m.predict(X_test)

# final prediction
y_stacked[:,-1] = np.mean(y_stacked[:,0:-1], axis=1)

In [ ]:
names = ('Linear regression', 'Ridge regression', 'Lasso regression', 'Elastic net', 'Random forest', 'Boosting', 'Stacked average')

for i, name in enumerate(names):
    score = np.sqrt(mean_squared_error(y_test, y_stacked[:,i]))
    print( "{:<20}{:.2f}".format(name, score) )

We can see that even a simple average performs better than the base models. Now, let's try the actual stacking. 

## 4.2 Hold-out stacking

This variant of stacking works as follow:
 - Set aside some of the training data, and train the base models on the remaining part. 
 - After the base models are trained, use them to generate the intermediate features from the hold-out data.
 - Train the meta learner using the intermediate features as inputs, and the hold-out labels as outputs.
 - On new data, use the base models to generate the intermediate predictions, and feed them to the meta learner.
 
---

**Exercise:** Implement stacking using the hold-out approach.

**Hint:** You can implement a class wrapping the base models and the meta learner into a single ensemble. The class has the following attributes and methods:
 - `base_models` - a list of scikit-learn models.
 - `meta_learner` - a single scikit-learn model.
 - `fit(X, y)` - Train the base models and the meta learner (see above).
 - `predict(X)` - Predict on new data (see above).

In [ ]:
class HoldOutStacking:
    
    def __init__(self, base_models, meta_learner):
        self.base_models = base_models
        self.meta_learner = meta_learner
    
    def fit(self, X, y):
        
        # 1. Split the data
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        
        ### YOUR CODE HERE ###
        
        # 2. Train the base models
        None
        
        # 3. Train the meta learner
        None
        
        ### END CODE HERE ###
        
        
    def predict(self, X):
        ### START CODE HERE ###
        
        # 1. Generate the intermediate features
        None
        
        # 2. Return the final predictions
        None
        
        ### END CODE HERE ###
        

In [ ]:
# stacking
stack = HoldOutStacking(models, meta_learner=LassoCV())

# train
stack.fit(X_train, y_train)

# test
y_pred = stack.predict(X_test)

# evaluate
score = np.sqrt(mean_squared_error(y_test, y_pred))

# show
print('Hold-out stacking: {:2.2f}'.format(score))

**Expected Output**:

<table style="width:30%">
  
  <tr>
    <td>**Hold-out stacking**</td>
    <td> 17867.76 </td> 
  </tr>
  
</table>

In [ ]:
np.testing.assert_almost_equal(score, 17867.76, decimal=0)

---

You can see that hold-out stacking is not better than stacked average. The reason is that the base models are trained on a smaller train set, as a portion of the original training data is set aside for the meta learner. Fortunately, this issue can be solved by using the K-fold approach.

## 4.2 K-fold stacking

This variant of stacking works as follow:
 - Divide the training data into K different folds. 
 - Train the base models using K-1 folds, and use them to make intermediate predictions from the unused fold. 
 - Repeat the process by holding out a different fold, and continue until you have generated the intermediate predictions for all the K folds.
 - Use all the training data to train the base models, and all the intermediate predictions to train the meta learner.
 - On new data, use the base models to generate the intermediate predictions, and feed them to the meta learner.
 
---

**Exercise:** Implement stacking using the K-fold approach.

**Hint:** You can implement a class wrapping the base models and the meta learner into a single ensemble. The class has the following attributes and methods:
 - `base_models` - a list of scikit-learn models.
 - `meta_learner` - a single scikit-learn model.
 - `fit(X, y)` - Train the base models and the meta learner (see above).
 - `predict(X)` - Predict on new data (see above).
 
You can use the scikit-learn function `cross_val_predict()` to generate the intermediate predictions.

In [ ]:
class KFoldStacking:
    
    def __init__(self, base_models, meta_learner):
        self.base_models = base_models
        self.meta_learner = meta_learner
    
    def fit(self, X, y):
        
        # 1. Define the K-folds
        cv = KFold(n_splits=5, random_state=42)
        
        ### YOUR CODE HERE ###
        
        # 2. Train the base models
        None
        
        # 3. Train the meta learner
        None
        
        ### END CODE HERE ###
        
        
    def predict(self, X):
        
        # 1. Generate the intermediate features
        X_meta = np.column_stack([model.predict(X) for model in self.base_models])
        
        # 2. Return the final predictions
        return self.meta_learner.predict(X_meta)

In [ ]:
# stacking
stack = KFoldStacking(models, meta_learner=LinearRegression())

# train
stack.fit(X_train, y_train)

# test
y_pred = stack.predict(X_test)

# evaluate
score = np.sqrt(mean_squared_error(y_test, y_pred))

# show
print('K-fold stacking: {:2.2f}'.format(score))

**Expected Output**:

<table style="width:30%">
  
  <tr>
    <td>**K-fold stacking**</td>
    <td> 17758.86 </td> 
  </tr>
  
</table>

In [ ]:
np.testing.assert_almost_equal(score, 17758.86, decimal=0)

---

You can see that K-fold stacking is slightly better than stacked average. However, there is still one thing that can be improved in our implementation. Remark that we train K different versions of the same base model to generate the intermediate features. This is how K-fold cross-validation works. However, we throw them away in the current implementation. Instead, we could make a better use of those models. Let's see how to do so.

## 4.3 Power stacking

This variant of stacking works as follow:
 - Divide the training data into K different folds. 
 - Train the base models using K-1 folds, and use them to make intermediate predictions on the unused fold. 
 - Repeat the process by holding out a different fold, and continue until you have ended up training K different versions of each base model, each of which generates the intermediate predictions for one of the K folds. 
 - Use the intermediate predictions to train the meta learner.
 - On new data, use the K ensembles of base models to generate K different sets of intermediate predictions, and average them together to get a single set, and feed it to the meta learner.

In [ ]:
class PowerStacking:
    
    def __init__(self, base_models, meta_learner):
        self.base_models = base_models
        self.meta_learner = meta_learner
    
    
    def fit(self, X, y):
        
        # 1. Initialize
        cv = KFold(n_splits=5, random_state=42)
        models = self.base_models
        self.base_models = [list() for x in models]
        
        # 2. Train the base models
        X_meta = np.zeros((X.shape[0], len(models)))
        for i, model in enumerate(models):
            for train_idx, valid_idx in cv.split(X, y):
                instance = clone(model)
                self.base_models[i].append(instance)
                instance.fit(X[train_idx,:], y[train_idx])
                X_meta[valid_idx, i] = instance.predict(X[valid_idx,:])
        
        # 3. Train the meta learner
        self.meta_learner.fit(X_meta, y)

        
    def predict(self, X):
        
        # 1. Generate the intermediate features
        X_meta = np.zeros((X.shape[0], len(self.base_models)))
        for i, base_models in enumerate(self.base_models):
            y_stacked = np.column_stack([model.predict(X) for model in base_models])
            X_meta[:,i] = np.mean(y_stacked, axis=1)
                    
        # 2. Return the final predictions
        return self.meta_learner.predict(X_meta)

In [ ]:
# stacking
stack = PowerStacking(models, meta_learner=LinearRegression())

# train
stack.fit(X_train, y_train)

# test
y_pred = stack.predict(X_test)

# evaluate
score = np.sqrt(mean_squared_error(y_test, y_pred))

# show
print('Power stacking: {:2.2f}'.format(score))

We can see that the performance has slightly improved. Let's also check the residual plot.

In [ ]:
plt.figure(figsize=(10,5))
sns.residplot(y_pred, y_test, lowess=True, color="r", scatter_kws={"color":"g", "s":20, "alpha": 0.4})
plt.title("Residual plot")

# 5. Conclusion

To recap, these are the performances obtained with the tested models. 

<table style="width:25%">

    <tr>
    <td>**Linear regression**</td>
    <td> 19880.33 </td> 
    </tr>
    
    <tr>
    <td>**Ridge regression**</td>
    <td> 18836.40 </td> 
    </tr>
    
    <tr>
    <td>**Lasso regression**</td>
    <td> 19044.91 </td> 
    </tr>
    
    <tr>
    <td>**Elastic net**</td>
    <td> 18980.21 </td> 
    </tr>
    
    <tr>
    <td>**Random forest**</td>
    <td> 20740.51 </td> 
    </tr>
    
    <tr>
    <td>**Boosting**</td>
    <td> 18512.43 </td> 
    </tr>
    
    <tr>
    <td>**Stacked average**</td>
    <td> 17835.13 </td> 
    </tr>
    
    <tr>
    <td>**Hold-out stacking**</td>
    <td> 17867.76 </td> 
    </tr>
    
    <tr>
    <td>**K-fold stacking**</td>
    <td> 17758.86 </td> 
    </tr>
    
    <tr>
    <td>**Power stacking**</td>
    <td> 17720.59 </td> 
    </tr>
  
</table>

We can see that stacking achieves the best performance on this dataset.